In [129]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib widget

from toolz import *

import os
import pickle
import json
import pathlib

import glom
from lenses import lens

In [2]:
lmap = compose(list, map)

In [3]:
def list_of_str_to_list_of_json(lst):
    it = map(lambda x: x.replace('\xa0', ' '), lst)
    it = filter(lambda x: x != '', it)
    it = lmap(json.loads, it)
    return it

In [4]:
data_path = pathlib.Path('data')

## Loading dataset from files (as nested structure):

In [9]:
dict_json_by_course = {}
for path_course in pathlib.Path('data').joinpath('collected').iterdir():
    list_of_course_jsons = []
    for path_file in path_course.iterdir():
        with open(path_file, 'r', encoding='utf-8') as f:
            list_of_course_jsons += list_of_str_to_list_of_json(f.readlines())
    dict_json_by_course[str(path_course.name)] = list_of_course_jsons

In [10]:
dict_json_by_course.keys()

dict_keys(['Выпечка и десерты', 'Завтраки', 'Закуски', 'Напитки', 'Основные блюда', 'Паста и пицца', 'Салаты', 'Соусы и маринады', 'Супы'])

In [14]:
instance_entry = dict_json_by_course['Завтраки'][0]
instance_entry

{'url': 'https://eda.ru/recepty/zavtraki/yaichnica-po-abhazski-138276',
 'title': 'Яичница по-абхазски',
 'category': 'Завтраки',
 'cuisine': 'Абхазская кухня',
 'menu': 'Еда без ограничений',
 'description': 'Рецепт взят из книги «Абхазская кухня» издательства «Эксмо».',
 'ingredients': [['Куриное яйцо', '3 штуки'],
  ['Аджика', '½ чайные ложки'],
  ['Сливочное масло', '15 г']],
 'instructions': [['1', 'Смешать яйца с аджикой, немного взбить.'],
  ['2',
   'На сковороде растопить сливочное масло и вылить на нее яичную смесь. Жарить яичницу 2–3 минуты, подавать горячей.']],
 'advice': ''}

## Processing:

Now we're ready to process this data to create dataset for our needs

In [18]:
def map_dict_of_lists(func, structure):
    """
    helper function to simplify mapping of 
    different functions to our structure
    `func` -- function that gets a dict as
        an input
    """
    res = valmap(
        lambda list_by_category: lmap(
            func,
            list_by_category),
        structure)
    return res

### First of all, renaming `category` to `course`:

In [42]:
dict_json_by_course_renamed = map_dict_of_lists(
    lambda instance: keymap(
        lambda field_name: 'course' if field_name == 'category' else field_name,
        instance),
    dict_json_by_course)
instance_entry = dict_json_by_course_renamed['Завтраки'][0]
instance_entry

{'url': 'https://eda.ru/recepty/zavtraki/yaichnica-po-abhazski-138276',
 'title': 'Яичница по-абхазски',
 'course': 'Завтраки',
 'cuisine': 'Абхазская кухня',
 'menu': 'Еда без ограничений',
 'description': 'Рецепт взят из книги «Абхазская кухня» издательства «Эксмо».',
 'ingredients': [['Куриное яйцо', '3 штуки'],
  ['Аджика', '½ чайные ложки'],
  ['Сливочное масло', '15 г']],
 'instructions': [['1', 'Смешать яйца с аджикой, немного взбить.'],
  ['2',
   'На сковороде растопить сливочное масло и вылить на нее яичную смесь. Жарить яичницу 2–3 минуты, подавать горячей.']],
 'advice': ''}

In [47]:
sample = {}
for i, key in enumerate(dict_json_by_course_renamed.keys()):
    sample[key] = []
    for j, lst_element in enumerate(dict_json_by_course_renamed[key]):
        sample[key].append(lst_element)
        sample[key][j]['ingredients'] = []
        sample[key][j]['instructions'] = []
        if j == 1: break
    if i == 2: break
sample

{'Выпечка и десерты': [{'url': 'https://eda.ru/recepty/vypechka-deserty/bananovoe-morozhenoe-s-koricej-28195',
   'title': 'Банановое мороженое с корицей',
   'course': 'Выпечка и десерты',
   'cuisine': 'Карибская кухня',
   'menu': 'Детское меню',
   'description': '',
   'ingredients': [],
   'instructions': [],
   'advice': ''},
  {'url': 'https://eda.ru/recepty/vypechka-deserty/ananasovo-kokosovij-sherbet-28141',
   'title': 'Ананасово-кокосовый шербет',
   'course': 'Выпечка и десерты',
   'cuisine': 'Карибская кухня',
   'menu': 'Еда без ограничений',
   'description': '',
   'ingredients': [],
   'instructions': [],
   'advice': ''}],
 'Завтраки': [{'url': 'https://eda.ru/recepty/zavtraki/yaichnica-po-abhazski-138276',
   'title': 'Яичница по-абхазски',
   'course': 'Завтраки',
   'cuisine': 'Абхазская кухня',
   'menu': 'Еда без ограничений',
   'description': 'Рецепт взят из книги «Абхазская кухня» издательства «Эксмо».',
   'ingredients': [],
   'instructions': [],
   'advic

In [121]:
target = {
    'system': {
        'planets': [
            {
                'name': 'earth',
                'moons': [
                    {'name': 'luna'}
                ]
            },
            {
                'name': 'jupiter',
                'moons': [
                    {'name': 'io'},
                    {'name': 'europa'}
                ]
            }
        ]
    }
}
spec = {
    'planet_names': ('system.planets', ['name']),
    'moon_names': ('system.planets', [('moons', ['name'])])
}
glom.glom(target, spec)

{'planet_names': ['earth', 'jupiter'],
 'moon_names': [['luna'], ['io', 'europa']]}

In [127]:
it = glom.glom(sample, ('*', ['*']))
(it)

[[{'url': 'https://eda.ru/recepty/vypechka-deserty/bananovoe-morozhenoe-s-koricej-28195',
   'title': 'Банановое мороженое с корицей',
   'course': 'Выпечка и десерты',
   'cuisine': 'Карибская кухня',
   'menu': 'Детское меню',
   'description': '',
   'ingredients': [],
   'instructions': [],
   'advice': ''},
  {'url': 'https://eda.ru/recepty/vypechka-deserty/ananasovo-kokosovij-sherbet-28141',
   'title': 'Ананасово-кокосовый шербет',
   'course': 'Выпечка и десерты',
   'cuisine': 'Карибская кухня',
   'menu': 'Еда без ограничений',
   'description': '',
   'ingredients': [],
   'instructions': [],
   'advice': ''}],
 [{'url': 'https://eda.ru/recepty/zavtraki/yaichnica-po-abhazski-138276',
   'title': 'Яичница по-абхазски',
   'course': 'Завтраки',
   'cuisine': 'Абхазская кухня',
   'menu': 'Еда без ограничений',
   'description': 'Рецепт взят из книги «Абхазская кухня» издательства «Эксмо».',
   'ingredients': [],
   'instructions': [],
   'advice': ''},
  {'url': 'https://eda.r

In [35]:
glom.glom(sample, '*.*.title')

[['Банановое мороженое с корицей', 'Ананасово-кокосовый шербет'],
 ['Яичница по-абхазски', 'Омлет с кукурузой'],
 ['Горячая закуска в багете', 'Быстрые закуски в тарталетках']]

In [ ]:
# with open('foodie_favorites_dataset/first_reviews.json', 'r') as f:
#     rewiews_lines = f.readlines() #json.load(f)

# rewiews_first = json.loads(rewiews_lines[1])

# with open('foodie_favorites_dataset/restaurants.json', 'r') as f:
#     for _ in range(1):
#         f.readline()
#     restaurant_nth = json.loads(f.readline())

# for i in range(len(rewiews_first['reviews'])):
#     print(rewiews_first['reviews'][i]['comment'])
#     print('\n' + '-'*100 + '\n')

list_vypechka = []
for file in data_path.joinpath('vypechka-deserty').iterdir():
    list_vypechka.append(file.read_text(encoding='utf-8'))

vypechka_path_to_add =  data_path.joinpath('data_file_vypechka_evropeyskaya.json')
with open(vypechka_path_to_add, 'r', encoding='utf-8') as f:
    list_vypechka += f.readlines()

vypechka_path_to_add =  data_path.joinpath('data_file.json')
with open(vypechka_path_to_add, 'r', encoding='utf-8') as f:
    list_vypechka += f.readlines()

for path in ['data_file_osnovnye_blyda.json',
             'data_file_Завтраки_Напитки_Паста_и_пицца_Соусы_и_маринады_Супы.json',
             'data_file_Закуски_Салаты.json']:
    path_obj = data_path.joinpath(path)
    with open(path_obj, 'r', encoding='utf-8') as f:
        path_list = f.readlines()
    json_form = json.dumps(list_of_str_to_list_of_json(list_vypechka), indent=2, ensure_ascii=False)
    path_obj.write_text(json_form)

json_form = json.dumps(list_of_str_to_list_of_json(list_vypechka), indent=2, ensure_ascii=False)
data_path.joinpath('vypechka-deserty.json').write_text(json_form)


In [34]:
res = {}
for elt in data_path.parent.joinpath('old').iterdir():
    if not elt.is_dir():
        res[str(elt)] = pd.json_normalize(json.loads(elt.read_text(encoding='utf-8')))

In [35]:
res_lst = list(res.values())
res_lst[0]

,title,category,cuisine,menu,description
0,Фаршированные белые грибы с сыром,Закуски,Русская кухня,Еда без ограничений,
1,"Рулеты из огурца, картофельного пюре и сельди",Закуски,Русская кухня,Еда без ограничений,
2,Глазированная репа,Закуски,Русская кухня,Еда без ограничений,
3,Баклажаны с грибами в сметанном соусе,Закуски,Русская кухня,Еда без ограничений,
4,Оладьи из малоглазого макруруса,Закуски,Русская кухня,Еда без ограничений,
...,...,...,...,...,...
7632,Мидии с пармезаном и белым вином,Салаты,Бельгийская кухня,Еда без ограничений,
7633,Салат с подкопченой семгой и хурмой,Салаты,Аргентинская кухня,Еда без ограничений,
7634,"Салат из мяса, овощей и моцареллы с соусом чим...",Салаты,Аргентинская кухня,Еда без ограничений,
7635,Биф-салат,Салаты,Аргентинская кухня,Еда без ограничений,
